In [ ]:
#	Prompt LLMs to be expert
#       they can now respond to question and provide structured response
#	Generate domain-specific multiple choice questions
#       on the way
#	Grade the result of LLM and find out correlations
#       with strcuture output, should be easy


In [12]:
import json
import io
import pandas as pd
from phi.agent import Agent, RunResponse
from phi.model.ollama import Ollama
from phi.storage.agent.sqlite import SqlAgentStorage
import requests

In [ ]:
# scrap the occupation file
cookies = {
    '_ga_CSLL4ZEK4L': 'GS1.1.1732201661.25.1.1732201725.0.0.0',
    '_ga': 'GA1.1.1544249262.1725888442',
    'nmstat': 'eff44f81-0b7b-6903-16db-900ccbff85c3',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Connection': 'keep-alive',
    'Referer': 'https://download.bls.gov/pub/time.series/oe/',
    # 'Cookie': '_ga_CSLL4ZEK4L=GS1.1.1732201661.25.1.1732201725.0.0.0; _ga=GA1.1.1544249262.1725888442; nmstat=eff44f81-0b7b-6903-16db-900ccbff85c3',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Priority': 'u=0, i',
    # Requests doesn't support trailers
    # 'TE': 'trailers',
}

response = requests.get('https://download.bls.gov/pub/time.series/oe/oe.occupation', cookies=cookies, headers=headers)

b'occupation_code\toccupation_name\toccupation_description\tdisplay_level\tselectable\tsort_sequence\r\n000000\tAll Occupations\t\t0\tT\t0\r\n110000\tManagement Occupations\t\t0\tT\t1\r\n111000\tTop Executives\t\t1\tT\t2\r\n111011\tChief Executives\tDetermine and formulate policies and provide overall direction of companies or private and public sector organizations within guidelines set up by a board of directors or similar governing body. Plan, direct, or coordinate operational activities at the highest level of management with the help of subordinate executives and staff managers.\t3\tT\t3\r\n111021\tGeneral and Operations Managers\tPlan, direct, or coordinate the operations of public or private sector organizations, overseeing multiple departments or locations. Duties and responsibilities include formulating policies, managing daily operations, and planning the use of materials and human resources, but are too diverse and general in nature to be classified in any one functional are

In [3]:
# global variables
model_id = "llama3.1"
#qwen2.5-coder, llama3.1, llama3.2


In [17]:
# Prompt LLMs to be expert

# get list of occupation
#https://www.bls.gov/oes/current/oessrci.htm
occupation = pd.read_csv(io.BytesIO(response.content), sep="\t").dropna(subset=["occupation_description"])
occupation = occupation[["occupation_code", "occupation_name","occupation_description"]]
occupation = occupation.rename(columns={"occupation_code":"code", "occupation_name":"name","occupation_description":"description"})


In [18]:
#occupation_l = ['New Accounts Clerks', 'Nuclear Technicians', 'Orderlies', 'Clinical and Counseling Psychologists', 'Radiologic Technologists and Technicians'] 
occupation_l = (list(occupation.sample(5).name))
sample_o = occupation.loc[occupation["name"].isin(occupation_l)]
sample_o

,code,name,description
145,172161,Nuclear Engineers,Conduct research on nuclear engineering projec...
246,211092,Probation Officers and Correctional Treatment ...,Provide social services to assist in rehabilit...
251,212011,Clergy,Conduct religious worship and perform other sp...
703,434151,Order Clerks,Receive and process incoming orders for materi...
834,491011,"First-Line Supervisors of Mechanics, Installer...",Directly supervise and coordinate the activiti...


In [6]:
#C.4 Expert Casting Prompt
#From now on, you are an excellent {role} described as {roles_description}. Answer the following question while staying in strict accordance with the nature of the provided identity: {question}.
#url = "https://arxiv.org/pdf/2411.00492"

def create_agent(role, role_description):
    expert_agent = Agent(
        model=Ollama(id=model_id),
        description="From now on, you are an excellent {} described as {}. you are tasked with answering multiple choice questions. Think through each step meticulously but only show me the final answer. Rules: 1.Think deeply about the question's core concept. 2. Consider common misconceptions. 3. Mentally justify why each wrong option is incorrect 4. Double-check your logic 5. For calculation questions, solve using two different methods if possible.".format(role, role_description),
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["-you are helpful and you think step by step."],
        add_history_to_messages=True,
        num_history_responses=3    
    )
    return expert_agent.run("what is your occupation").content, expert_agent.session_id

In [7]:
agents_in_action = {}
for role in range(len(sample_o)):
    agents_in_action[sample_o["name"].iloc[role]] = create_agent(sample_o["name"].iloc[role], sample_o["description"].iloc[role])[1]
agents_in_action

{'Agricultural Engineers': '049e054e-7246-475f-87ea-7e9d9963e710',
 'Telephone Operators': '826906a4-e151-4e30-a0d3-916334155f75',
 'Fallers': 'cc721bb3-8685-4bb1-a4b2-a61dcc80d10c',
 'Helpers--Painters, Paperhangers, Plasterers, and Stucco Masons': '1dc41704-6e0d-489e-b736-8dcde47e5dbc',
 'Crushing, Grinding, and Polishing Machine Setters, Operators, and Tenders': 'b1b7f33c-dfe2-47b2-bd82-095706092fec'}

In [23]:
agent_list =list(agents_in_action.keys())
agent_list

['Agricultural Engineers',
 'Telephone Operators',
 'Fallers',
 'Helpers--Painters, Paperhangers, Plasterers, and Stucco Masons',
 'Crushing, Grinding, and Polishing Machine Setters, Operators, and Tenders']

In [22]:
question = "Question: A ball is thrown vertically upward with an initial velocity of 20 m/s. Assuming g = 10 m/s², what is the maximum height reached by the ball? Options: A) 10 meters B) 20 meters C) 30 meters D) 40 meters Your answer: [Option]"


In [35]:
#-you structure your output in a dictionary {"question number": "answer option"}. for example: {"1": "A", "2":"C", "3":"C"}
def ask_mc(question, session):
    expert_agent = Agent(
        model=Ollama(id=model_id),
        description="you are tasked with answering multiple choice questions. Think through each step meticulously but only show me the final answer. Rules: 1.Think deeply about the question's core concept. 2. Consider common misconceptions. 3. Mentally justify why each wrong option is incorrect 4. Double-check your logic 5. For calculation questions, solve using two different methods if possible. for output format, do this: [option (A, B, D, or D. do not repeat the answer text. alphbet only)]",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["""-you think step by step. -do not provide explanations. -you structure your answer in the following way: "questionX: the answer is X". """],
        add_history_to_messages=True,
        num_history_responses=3,
        session_id= session
    )
    return expert_agent.run(question).content

In [ ]:
# for agent in agents_in_action:
#     print(ask_mc(question, agents_in_action[agent]))

('The answer is D) Yellow.', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion1: the answer is C', '80b2d652-2772-4d44-8c28-1775e7f8cfc4')
('qustion1: the answer is C', 'f40c78f5-3a71-40af-b3c9-db9be14cdd06')
('C) Green', 'd652f109-ddf6-4c34-ba9d-6034885217e0')
('qustion1: the answer is C', 'bbd0b655-5515-4ce8-8f8c-1f83977178d8')


In [ ]:
# question = """Answer this multiple choice question. Think through each step meticulously but only show me the final answer.

# Question: In a population of 200 students, 60% are female. How many male students are in the population?

# Options:
# A) 80 students
# B) 120 students
# C) 140 students
# D) 160 students"""

In [ ]:
# question = """Question: Which of the following is NOT a primary color in traditional color theory?

# Options:
# A) Red
# B) Blue
# C) Green
# D) Yellow"""

In [ ]:
# with open("questions.json", "r") as file:
#     qb = json.load(file)

# for q in range(len(qb)):
#     print(qb[q]["question"],qb[q]["options"] )

# questions = [[qb[q]["question"],qb[q]["options"]] for q in range(len(qb))]
# answers = [qb[q]["answer"] for q in range(len(qb))]
# questions

When positioning a patient for a chest X-ray, which adjustment ensures the clavicles are out of the lung field? ['Angle the X-ray tube cephalad.', 'Position the patient in a prone position.', 'Align the X-ray beam perpendicular to the sternum.', 'Ask the patient to roll their shoulders forward.']
What is the recommended protocol when imaging a pediatric patient who cannot remain still? ['Use sedation to keep the patient immobile.', 'Use the shortest possible exposure time and immobilization devices.', 'Allow a parent to hold the child during the procedure without protective gear.', 'Perform the imaging procedure despite movement for efficiency.']
A fluoroscopy exam shows unusually high patient skin dose readings. What should you do? ['Stop the procedure immediately and report the incident.', 'Continue the procedure but decrease the fluoroscopy time.', 'Confirm equipment calibration and assess for overexposure risks.', 'Lower the table to reduce the distance between the X-ray source and

[['When positioning a patient for a chest X-ray, which adjustment ensures the clavicles are out of the lung field?',
  ['Angle the X-ray tube cephalad.',
   'Position the patient in a prone position.',
   'Align the X-ray beam perpendicular to the sternum.',
   'Ask the patient to roll their shoulders forward.']],
 ['What is the recommended protocol when imaging a pediatric patient who cannot remain still?',
  ['Use sedation to keep the patient immobile.',
   'Use the shortest possible exposure time and immobilization devices.',
   'Allow a parent to hold the child during the procedure without protective gear.',
   'Perform the imaging procedure despite movement for efficiency.']],
 ['A fluoroscopy exam shows unusually high patient skin dose readings. What should you do?',
  ['Stop the procedure immediately and report the incident.',
   'Continue the procedure but decrease the fluoroscopy time.',
   'Confirm equipment calibration and assess for overexposure risks.',
   'Lower the table

In [26]:
prompt = """please design 10 multiple choice questions for this occupation. some thoughts before you repond: 
1. be domain specific. ideally design for someone with more than 10 year experience.
2. create some nuance. make the answers somewhat similar so that people cannot guess by intuition.
3. do not limit yourself with only factual question. be creative.
4. output your questions in json format where new line is strickly prohitbited. only produce it in compact format. sample format: 
[{"question":"When positioning a patient for a chest X-ray, which adjustment ensures the clavicles are out of the lung field?","options":["Angle the X-ray tube cephalad.","Position the patient in a prone position.","Align the X-ray beam perpendicular to the sternum.","Ask the patient to roll their shoulders forward."],"answer":"d","reason":"Rolling shoulders forward moves the clavicles out of the lung field."},{"question":"What is the recommended protocol when imaging a pediatric patient who cannot remain still?","options":["Use sedation to keep the patient immobile.","Use the shortest possible exposure time and immobilization devices.","Allow a parent to hold the child during the procedure without protective gear.","Perform the imaging procedure despite movement for efficiency."],"answer":"b","reason":"Short exposure time and immobilization reduce radiation and ensure safety."}]
5. only output the required json. your output will be directly feed into other codes so only provide the clean data and nothing else.
here is the occupation: """

In [27]:
def mc_magician(role):
    mc_agent = Agent(
        model=Ollama(id=model_id),
        description=""""youre someone with these qualities:
1. Deep Subject Matter Expertise
    Extensive knowledge and experience in the specific domain, ensuring that the questions test practical and theoretical knowledge relevant to the field.
    Familiarity with industry trends, tools, and challenges to create current and applicable scenarios.

2. Pedagogical Skills
    Ability to design questions that evaluate not only memorized knowledge but also problem-solving, analytical thinking, and applied skills.
    Experience in educational design or certification processes.

3. Understanding of Professional Requirements
    Awareness of the key competencies required for success in the profession. This could involve collaborating with industry professionals or studying job requirements.
    Ability to assess levels of proficiency, from beginner to expert.

4. Analytical and Research Skills
    Capable of analyzing job roles and breaking them into assessable skills and knowledge areas.
    Conducting research to stay updated on technological or methodological advancements.

5. Collaboration with Industry Practitioners
    Collaboration with professionals currently working in the field to ensure questions reflect real-world challenges and practices.
    Incorporating feedback from companies or organizations that will use the certification.

6. Access to Validation Tools
    Familiarity with tools for benchmarking and validating questions to ensure they are neither too simple nor overly complex.
    Experience with psychometric techniques to gauge question effectiveness.""",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["you only output what is reuqired and never provide explanation or additional information."],
        add_history_to_messages=True,
        num_history_responses=3,
        
    )
    return mc_agent.run(prompt+role).content, mc_agent.session_id

In [28]:
mc_magician(agent_list[0])

('[{"question":"What is the primary consideration when designing an irrigation system for a farm with high soil salinity?","options":["Maximize water flow rate to ensure crop health.","Implement a drip irrigation system to reduce evaporation.","Use a sprinkler system to promote soil aeration.","Design a flood irrigation system to increase land use efficiency."],"answer":"b","reason":"Drip irrigation reduces evaporation and prevents further soil salinization."},{"question":"A farm is considering using anaerobic digesters to produce biogas from agricultural waste. What should be the primary concern when siting these facilities?","options":["Minimizing land use and visual impact.","Ensuring adequate space for future expansion.","Maximizing proximity to the main farmhouse for convenience.","Protecting nearby water sources due to ammonia emissions."],"answer":"d","reason":"Anaerobic digesters emit ammonia, which can contaminate water sources."},{"question":"What is the recommended approach 

In [29]:
result = mc_magician(agent_list[0])
qb = json.loads(result[0])
for q in range(len(qb)):
    print(qb[q]["question"],qb[q]["options"] )

questions = [[qb[q]["question"],qb[q]["options"]] for q in range(len(qb))]
answers = [qb[q]["answer"] for q in range(len(qb))]
questions

What is the primary goal of precision irrigation systems in agricultural engineering? ['Reducing water usage to minimize costs.', 'Increasing crop yield through optimized water distribution.', 'Enhancing soil aeration for better root growth.', 'Improving air circulation within greenhouses for temperature control.']
Which of the following materials is most suitable for constructing a greenhouse with a high humidity environment? ['Aluminum alloys due to their corrosion resistance.', 'Stainless steel pipes for durability and low maintenance.', 'PVC-coated fabric for water resistance and UV stability.', 'Galvanized iron sheets for cost-effectiveness.']
What is the purpose of implementing a crop rotation system as part of an agricultural engineering project? ['To reduce soil erosion by adding organic matter.', 'To increase plant disease transmission by spreading pathogens.', 'To enhance soil fertility through nutrient cycling.', 'To decrease biodiversity by promoting monoculture.']
Which ag

[['What is the primary goal of precision irrigation systems in agricultural engineering?',
  ['Reducing water usage to minimize costs.',
   'Increasing crop yield through optimized water distribution.',
   'Enhancing soil aeration for better root growth.',
   'Improving air circulation within greenhouses for temperature control.']],
 ['Which of the following materials is most suitable for constructing a greenhouse with a high humidity environment?',
  ['Aluminum alloys due to their corrosion resistance.',
   'Stainless steel pipes for durability and low maintenance.',
   'PVC-coated fabric for water resistance and UV stability.',
   'Galvanized iron sheets for cost-effectiveness.']],
 ['What is the purpose of implementing a crop rotation system as part of an agricultural engineering project?',
  ['To reduce soil erosion by adding organic matter.',
   'To increase plant disease transmission by spreading pathogens.',
   'To enhance soil fertility through nutrient cycling.',
   'To decrea

In [32]:
answers

['b', 'c', 'c', 'c', 'a', 'a', 'd', 'd', 'c', 'd']

In [33]:
def answer_cleaner(input):
    clean_agent = Agent(
        model=Ollama(id=model_id),
        description="you are a profession computer programmer with 20 years of experience. you are good at cleaning data and cextract answer to be a list of strings. you will receive input, and please extract answer from it. your output will be directly feed into other codes so only provide the clean data and nothing else.example: ['x','x','x','x','x','x'] where 'x' is either D, B, C, or A",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["you only output what is reuqired and never provide explanation or additional information"],
        add_history_to_messages=True,
        num_history_responses=3,
        
    )
    return clean_agent.run(input).content

In [37]:
for agent in agents_in_action:
    for q in range(len(questions)):
        print(ask_mc("""format is very important. remember to structure your result. no need to provide reaons. here is the question:"""+ str(questions[q]), agents_in_action[agent]))


What is the primary goal of precision irrigation systems in agricultural engineering?: A
Which of the following materials is most suitable for constructing a greenhouse with a high humidity environment?: C
What is the purpose of implementing a crop rotation system as part of an agricultural engineering project?: C
Which agricultural machine component is most susceptible to wear due to its constant exposure to soil and crop debris?: B
What is the primary benefit of integrating solar power into an agricultural operation?: A
Which agricultural engineering system would benefit most from the integration of advanced sensing technologies?: C
What is the primary objective of implementing a drip irrigation system in an agricultural engineering project?: D
Which of the following agricultural engineering systems is most vulnerable to cyber attacks?: D
What is the primary purpose of integrating a composting system in an agricultural operation?: C
Which of the following agricultural engineering pro

In [ ]:
b,c,c,b,a,c,d,d,c,c

In [190]:
store

[('', '7b0ea7be-49be-480f-a810-95a529125472'),
 ('', '1f601db4-7f71-424e-87a8-a96a9aa377bd'),
 ('', '1a50066f-ee7c-4796-b8fe-8cae71f1c902'),
 ('', '70be8b5a-5898-4d71-a3ee-eb2bac0efc2a'),
 ('', '574971a1-d045-4c91-ba3e-570d115f9149'),
 ('', 'ab793c5e-a807-4dd0-8741-37a051234e74'),
 ('', '6e4a4542-1d18-4d69-b105-528967ca9a3c'),
 ('', '34f921f5-fb68-417d-b070-4db079d67982'),
 ('', 'fc11e6fe-7793-4582-9c0c-b207d35d1699'),
 ('', '4d899baa-2aff-4eaf-ad6f-7291615b01a0'),
 ('', 'fe6c81c3-9e50-4360-b052-2605a7d00fb5'),
 ('', 'f8a1bf45-77b9-45da-953a-623b25943a41'),
 ('', '7ef9f6de-1874-4c0b-86ce-254016bc83c6'),
 ('', '41a45682-7b44-47bd-8cc8-d12224d0fa79'),
 ('', '010500fa-c6f4-4d5f-8066-7aa0507a2508'),
 ('', 'fe4b9e27-e576-46c5-8a94-13280988aa37'),
 ('', '66d3297a-b3b2-4654-8b25-fe4bfe7b4bba'),
 ('', 'b2c1a450-3ce7-4208-b184-983c59cb3ee5'),
 ('', '41e50342-623c-4ba4-962b-215670462f50'),
 ('', '5ce98f1e-890e-4d68-bba5-0fc8e509ff29'),
 ('', '48c8077a-9804-4c04-bec7-9d623d694734'),
 ('', '377753

In [1]:
messydata = """('qustion:1, answer:A', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:2, answer:B', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:3, answer:C', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:4, answer:C', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:5, answer:C', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:6, answer:B', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:7, answer:C', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:8, answer:D', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:9, answer:A', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:10, answer:B', '39f281cb-5653-4c53-aa71-93693110ec63')"""

In [ ]:
eval(answer_cleaner(messydata)[0])
#use json.loads

['A', 'B', 'C', 'C', 'C', 'C', 'B', 'C', 'D', 'A', 'B']